## Import Libraries

In [25]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [26]:
import matplotlib.pyplot as plt
plt.ion()

In [27]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [28]:
from os import getcwd, path, pardir

## Compile and program project

In [29]:
project_path = [path.curdir, "No_Security_Flash"]
project_path

['.', 'No_Security_Flash']

In [89]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/nvmctrl" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.109\include" -I"../../../../shared"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: No_Security_Flash.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolc

0

## Data Logging

In [48]:
live_plot = False

Create a figure for the plot.

In [49]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [50]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_no_security_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment No Security",
}

Stop criteria to pass to the logger:

In [51]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [90]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

# Store Data

In [53]:
import pickle
pickle.dump(data, open("no_security_flash_logger_data.p", "wb"))

# Load Data

In [54]:
data = pickle.load(open("no_security_flash_logger_data.p", "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [55]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [91]:
flash_write_charge, flash_write_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [92]:
flash_read_charge, flash_read_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [93]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(flash_write_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(flash_write_charge) - 1}")

MAX_NUM_BYTES: 600


In [94]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [95]:
results = []
for y in [flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    fig, grid = result.plot()
#     xlabel=f"{labels[result_type]['x']} " +
#            f"[{labels[result_type]['x_unit']}]",
#     ylabel=f"{labels[result_type]['y']} " +
#            f"[{labels[result_type]['y_unit']}]")
#     fig.canvas.set_window_title(
#         f"Residuals of {parameter_name}")
    fig.tight_layout()
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 600
    # variables        = 2
    chi-square         = 1.2417e-09
    reduced chi-square = 2.0764e-12
    Akaike info crit   = -16138.2261
    Bayesian info crit = -16129.4322
[[Variables]]
    slope:      3.0995e-08 +/- 3.3964e-10 (1.10%) (init = 0)
    intercept:  2.9906e-06 +/- 1.1780e-07 (3.94%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.866

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 600
    # variables        = 2
    chi-square         = 6.3492e-14
    reduced chi-square = 1.0617e-16
    Akaike info crit   = -22066.8780
    Bayesian info crit = -22058.0841
[[Variables]]
    slope:      1.4508e-09 +/- 2.4287e-12 (0.17%) (init = 0)
    intercept:  7.1860e-09 +/- 8.4237e-10 (11.72%) (init = 1)
[[Correlations]] (unrepo

In [85]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title("Analysis No Security Flash")

In [96]:
charge_color = 'r'
time_color = 'b'

In [97]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [98]:
lines = []
lines += ax1.plot(num_bytes, flash_write_charge, charge_color+'-', label='Flash Write Charge')
lines += ax1.plot(num_bytes, flash_read_charge, charge_color+'--', label='Flash Read Charge')
lines += ax2.plot(num_bytes, flash_write_time, time_color+'-', label='Flash Write Time')
lines += ax2.plot(num_bytes, flash_read_time, time_color+'--', label='Flash Read Time')
ax1.legend(handles=lines)
# [flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
ax1.set_title(f"Flash Write Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} C per byte\n" +
             f"Flash Read Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} C per byte\n" +
             f"Flash Write Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"Flash Read Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [65]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(flash_write_charge, open("no_security_flash_write_charge.p", "wb"))
pickle.dump(flash_read_charge, open("no_security_flash_read_charge.p", "wb"))
pickle.dump(flash_write_time, open("no_security_flash_write_time.p", "wb"))
pickle.dump(flash_read_time, open("no_security_flash_read_time.p", "wb"))

## Write config file

In [99]:
import json

config = {}
config["name"] = "No Security Flash"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Flash Write"], 3: ["Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["Flash Write"],
                                        "load": ["Flash Read"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": 1}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)